In [17]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import u_V, u_Ohm, u_F, u_ns, u_ms, u_s

In [14]:
def simulate_rc_transient(R_value, C_value, V_in):
    """
    Simule la réponse transitoire d'un circuit RC à une excitation en échelon.
    Retourne les instants, la tension d'entrée, la tension de sortie (Vout),
    la tension sur la résistance (V_R) et sur le condensateur (V_C).
    """
    circuit = Circuit('RC Transient')
    circuit.PulseVoltageSource('input', 'vin', circuit.gnd,
                           initial_value=0@u_V, pulsed_value=V_in@u_V,
                           delay_time=0@u_s, rise_time=1@u_ns,
                           fall_time=1@u_ns, pulse_width=1@u_s, period=2@u_s)
    circuit.R(1, 'vin', 'vout', R_value @ u_Ohm)
    circuit.C(1, 'vout', circuit.gnd, C_value @ u_F)

    simulator = circuit.simulator(temperature=25, nominal_temperature=25)

    end = 5*R_value*C_value
    step = end/100
    analysis = simulator.transient(step_time=step, end_time=end)

    time = np.array([float(t) for t in analysis.time])
    v_in_obs = np.array([float(analysis['vin'][i]) for i in range(len(time))])
    v_out_obs = np.array([float(analysis['vout'][i]) for i in range(len(time))])

    return time, v_in_obs, v_out_obs

In [15]:
r_values = np.linspace(1e3, 22e3, 22)
c_values = np.linspace(1e-8, 2e-6, 22)
v_in = np.linspace(1, 22, 22)

In [22]:
open("rc_dataset_transient_complet.csv", "w").close()

with open("rc_dataset_transient_complet.csv", "w", newline='') as file:
    fieldnames = ['R', 'C', 'Time', 'Vin', 'Vout']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for R in r_values:
        for C in c_values:
            for V in v_in:
                t_BD, v_in_BD, v_out_BD = simulate_rc_transient(R, C, V)
                for t, vin, vout in zip(t_BD, v_in_BD, v_out_BD):
                    # Calcul analytique
                    vout_theo = vin * (1 - np.exp(-t / (R * C)))

                    # Vérification (exemple de seuil à 10%)
                    if abs(vout - vout_theo) > 0.01 * V:
                        print(f"Simulation : R={R}, C={C}, Vin={V}, t={t:.4e}")
                        print(f"  Vin simulé = {vin:.4f}")
                        print(f"  Vout simulé = {vout:.4f}, Vout théorique = {vout_theo:.4f}")
                    writer.writerow({
                        'R': R,
                        'C': C,
                        'Time': t,
                        'Vin': vin,
                        'Vout': vout
                    })

print(sum(1 for _ in open("rc_dataset_transient_complet.csv")) - 1)

1394606
